<a href="https://colab.research.google.com/github/midhun-james/val-mod-with-gliner/blob/main/rag_gliner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gliner
from gliner import GLiNER

# Load and cache the model
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

# Save the model locally
model.save_pretrained("gliner_model")

# Zip everything needed
!mkdir gliner_package
!cp -r /usr/local/lib/python3.*/dist-packages/gliner gliner_package/
!cp -r gliner_model gliner_package/
!zip -r gliner_package.zip gliner_package

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.8 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


  adding: gliner_package/ (stored 0%)
  adding: gliner_package/gliner_model/ (stored 0%)
  adding: gliner_package/gliner_model/special_tokens_map.json (deflated 50%)
  adding: gliner_package/gliner_model/tokenizer.json (deflated 77%)
  adding: gliner_package/gliner_model/added_tokens.json (deflated 36%)
  adding: gliner_package/gliner_model/spm.model (deflated 50%)
  adding: gliner_package/gliner_model/gliner_config.json (deflated 64%)
  adding: gliner_package/gliner_model/pytorch_model.bin (deflated 10%)
  adding: gliner_package/gliner_model/tokenizer_config.json (deflated 79%)
  adding: gliner_package/gliner/ (stored 0%)
  adding: gliner_package/gliner/__pycache__/ (stored 0%)
  adding: gliner_package/gliner/__pycache__/model.cpython-312.pyc (deflated 57%)
  adding: gliner_package/gliner/__pycache__/utils.cpython-312.pyc (deflated 36%)
  adding: gliner_package/gliner/__pycache__/config.cpython-312.pyc (deflated 41%)
  adding: gliner_package/gliner/__pycache__/__init__.cpython-312.pyc

In [1]:
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 48.6 MB/s eta 0:00:00


In [2]:
pip install gliner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00


In [15]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import gzip
import json
from gliner import GLiNER
import re
from collections import defaultdict

g_model=GLiNER.from_pretrained("urchade/gliner_medium-v2.1")
patterns = {
    'email': r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b',
    'phone': r'\b(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{2,4}\)?[-.\s]?)?\d{3,4}[-.\s]?\d{4}\b',
    'id': r'\b[A-Z]{5}[0-9]{4}[A-Z]\b'
}

labels = ["name of a person", "organization","IT service"]
with gzip.open("faker_dataset_v3.json.gz", "rt", encoding="utf-8") as f:
    fake_list= json.load(f)
fake_data={}
for d in fake_list:
    fake_data.update(d)
entity_column_map={"name of a person":"names",
                   "organization":"company"}
forward_mapping=defaultdict(dict)
backward_mapping=defaultdict(dict)
entity_to_fake={}
used_fakes={"names": set(), "company": set()}
def chunk_text_no_split(text, max_len=390, overlap_words=3):
    """
    Chunk text into pieces of max_len, ensuring no word is split between chunks.
    Overlap is handled by number of words, not characters.
    Returns a list of (chunk, start_char_index) tuples.
    """
    words = re.findall(r'\S+', text)
    chunks = []
    start = 0
    while start < len(words):
        end = start
        char_count = 0
        chunk_words = []
        while end < len(words) and char_count + len(words[end]) + (1 if chunk_words else 0) <= max_len:
            chunk_words.append(words[end])
            char_count += len(words[end]) + (1 if chunk_words else 0)
            end += 1
        chunk_text = ' '.join(chunk_words)
        # Find the character index in the original text
        if chunk_words:
            first_word = chunk_words[0]
            start_char = text.find(first_word, 0 if not chunks else chunks[-1][1] + 1)
        else:
            start_char = 0
        chunks.append((chunk_text, start_char))
        if end == len(words):
            break
        # Overlap by words
        start = end - overlap_words if end - overlap_words > 0 else end
    return chunks
# --- Step 1: Prepare Documents ---
def extract_entities_no_spacy(text):
    regex_entities = []
    for label, pattern in patterns.items():
        for match in re.finditer(pattern, text):
            regex_entities.append({
                'text': match.group(),
                'start': match.start(),
                'end': match.end(),
                'label': label
            })
    # Directly use the full text for GLiNER
    chunks = chunk_text_no_split(text, max_len=384, overlap_words=3)
    all_entities = []
    for chunk_text_part, offset in chunks:
        ents = g_model.predict_entities(chunk_text_part, labels, threshold=0.5)
        # If you want to map positions, you can do so here, but since we're not filtering, just offset
        for ent in ents:
            all_entities.append({
                'text': ent['text'],
                'start': ent['start'] + offset,
                'end': ent['end'] + offset,
                'label': ent['label']
            })
    all_entities.sort(key=lambda x: x['start'])
    return all_entities
def get_fake_value(label, real_value):
    # Map label to key
    if label.lower() == "name of a person":
        key = "names"
    elif label.lower() == "organization":
        key = "company"
    else:
        return real_value  # No masking for unknown labels

    pool = fake_data.get(key, [])

    # Check if already mapped in forward_mapping
    if real_value in forward_mapping[key]:
        return forward_mapping[key][real_value]

    # Find unused fake value
    for fake in pool:
        if fake not in used_fakes[key]:
            used_fakes[key].add(fake)

            # Update forward and backward mappings
            forward_mapping[key][real_value] = fake
            backward_mapping[key][fake] = real_value

            return fake

    return real_value
def mask_sentence(sentence):

        flat_map = {}
        for entity, value_map in forward_mapping.items():
            for original, fake in value_map.items():
                flat_map[original] = fake
        # Pre-lowercased lookup for fast replacement
        flat_map_lower = {k.lower(): v for k, v in flat_map.items()}
        # Identify which keys are present in the sentence (case-insensitive)
        sentence_lower = sentence.lower()
        matched_keys = [k for k in flat_map if k.lower() in sentence_lower]

        if matched_keys:
            # Sort matched keys by length (longest first) to avoid partial replacement
            matched_keys.sort(key=len, reverse=True)

            # Build regex pattern with alternation

            pattern = re.compile(
                r'(?<!\w)([\{\(\["\'\*\_]*?)(' +
                '|'.join(re.escape(k) for k in matched_keys) +
                r')([\}\)\]"\'\*\_]*?)(?!\w)',flags=re.IGNORECASE
            )
            def replace_match(match):
                prefix = match.group(1)  # e.g., '{' or '**'
                core = match.group(2)    # e.g., 'abc'
                suffix = match.group(3)  # e.g., '}' or '**'

                replaced = flat_map_lower.get(core.lower(), core)
                # print(f'{match.group(0)} => {prefix}{replaced}{suffix}')
                return f"{prefix}{replaced}{suffix}"
            sentence = pattern.sub(replace_match, sentence)

        return sentence
def unmask_summary(sentence):
        flat_map = {}
        for entity, value_map in backward_mapping.items():
            for original, fake in value_map.items():
                flat_map[original] = fake
        # Pre-lowercased lookup for fast replacement
        flat_map_lower = {}
        for fake,original in flat_map.items():
            fake_lower=fake.lower()
            flat_map_lower[fake_lower]=original

            core=re.sub(r'\b(co|llc|inc|group|international|corporation|ltd|)\.?$', '', fake_lower, flags=re.IGNORECASE).strip()
            if core and core!= fake_lower:
                flat_map_lower[core]=original
        # Identify which keys are present in the sentence (case-insensitive)
        sentence_lower = sentence.lower()
        matched_keys = [k for k in flat_map_lower if k in sentence_lower]

        if matched_keys:
            # Sort matched keys by length (longest first) to avoid partial replacement
            matched_keys.sort(key=len, reverse=True)

            # Build regex pattern with alternation
            pattern = re.compile(
                r'(?<!\w)([\{\(\["\'\*\_]*?)(' +
                '|'.join(re.escape(k) for k in matched_keys) +
                r')([\}\)\]"\'\*\_]*?)(?!\w)',flags=re.IGNORECASE
            )

            def replace_match(match):
                prefix = match.group(1)  # e.g., '{' or '**'
                core = match.group(2)    # e.g., 'abc'
                suffix = match.group(3)  # e.g., '}' or '**'

                replaced = flat_map_lower.get(core.lower(), core)
                # print(f'{match.group(0)} => {prefix}{replaced}{suffix}')
                return f"{prefix}{replaced}{suffix}"

            sentence = pattern.sub(replace_match, sentence)

        return sentence


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
string = """1. James Carter works at TechNova Solutions as a Senior Cloud Engineer.
2. Maya Patel is a Lead Data Scientist at NeoAI Labs.
3. Michael Zhang is the Program Manager at CloudSphere Technologies.
4. Sophia Rodriguez is Director of Cybersecurity at Sentinel Secure.
5. David Kim works as a Senior DevOps Specialist at SkyGrid IT.
6. Emma Johnson is a Business Analyst at DataBridge Consulting.
7. Rajiv Menon serves as CTO of QuantumCore Systems.
8. Olivia Brown leads the AI Product Strategy division at QuantumCore Systems.
9. Ethan Hughes is a Solutions Architect at QuantumCore Systems.
10. Isabella Chen is a Machine Learning Engineer at InnovateX Labs.
11. TechNova Solutions provides cloud-native SaaS platforms.
12. NeoAI Labs specializes in zero-shot classification research.
13. CloudSphere Technologies develops machine learning pipelines.
14. Sentinel Secure builds real-time threat detection services.
15. SkyGrid IT manages container orchestration systems.
16. DataBridge Consulting provides ETL pipelines and data lakes.
17. QuantumCore Systems focuses on multi-tenant architecture.
18. InnovateX Labs researches NLP-based knowledge graphs.
19. James Carter attended a cloud computing summit in Singapore.
20. Maya Patel spoke on AI-driven analytics for enterprise solutions.
21. Michael Zhang presented compliance frameworks like ISO 27001.
22. Sophia Rodriguez discussed SOC 2 security certifications.
23. Warner Kim explained Kubernetes and serverless functions.
24. Emma Johnson led requirement gathering sessions for AI models.
25. Rajiv Menon emphasized edge computing in predictive dashboards.
26. Olivia Brown focused on API-first integrations for SaaS.
27. Ethan Hughes designed microservices and event-driven systems.
28. Isabella Chen demonstrated customer support automation tools.
29. The team proposed real-time performance monitoring dashboards.
30. Experts collaborated to integrate intelligent data pipelines.
31. A project goal was to enhance enterprise-level decision-making.
32. Cloud-native architecture ensured scalability and resilience.
33. AI ethics and IT compliance were key focus areas.
34. Threat detection was powered by anomaly detection algorithms.
35. Machine learning pipelines processed millions of data points.
36. Zero-shot classification improved context understanding.
37. NLP-based knowledge graphs linked key organizational entities.
38. Predictive maintenance dashboards prevented downtime.
39. Edge computing accelerated response times.
40. Multi-tenant architecture supported large-scale deployments.
41. ISO 27001 ensured information security management compliance.
42. SOC 2 guaranteed data protection standards.
43. DevOps automation improved software release cycles.
44. Kubernetes simplified container management at scale.
45. Serverless functions reduced operational costs.
46. Data lakes stored structured and unstructured data.
47. ETL processes transformed raw logs into analytics-ready data.
48. Real-time anomaly detection identified security risks early.
49. Cybersecurity teams monitored potential breaches proactively.
50. CloudSphere used AI-based risk scoring for banking clients.
51. Sentinel Secure offered identity and access management services.
52. SkyGrid IT enabled seamless hybrid cloud deployments.
53. DataBridge Consulting advised on compliance-driven architectures.
54. InnovateX Labs built AI prototypes for enterprise chatbots.
55. QuantumCore Systems researched generative AI integrations.
56. Maya Patel collaborated with James Carter on cloud security.
57. Michael Zhang coordinated tasks among cross-functional teams.
58. Sophia Rodriguez conducted penetration testing simulations.
59. David Kis implemented CI/CD pipelines for faster delivery.
60. Emma Johnson analyzed business KPIs to refine ML models.
61. Rajiv Menon invested in multi-cloud data orchestration.
62. Olivia Brown championed responsible AI practices.
63. Ethan Hughes built RESTful APIs for third-party integration.
64. Isabella Chen integrated zero-shot learning in chatbot models.
65. AI services scaled across multiple geographic regions.
66. Real-time dashboards visualized system health metrics.
67. Edge computing reduced latency for end-users.
68. Data governance ensured audit-ready compliance.
69. Machine learning pipelines included anomaly detection modules.
70. Cloud-native SaaS optimized resource utilization.
71. Security audits validated SOC 2 readiness.
72. DevOps engineers improved rollback procedures.
73. Kubernetes clusters were deployed across cloud providers.
74. Serverless APIs handled unpredictable traffic surges.
75. ETL pipelines maintained schema consistency.
76. Predictive maintenance minimized downtime risk.
77. Knowledge graphs linked customer and product data.
78. AI-driven chatbots improved response accuracy.
79. API-first design accelerated development cycles.
80. NLP models identified customer intent efficiently.
81. Business analysts tracked performance using KPIs.
82. Solutions architects planned system migrations.
83. CTOs prioritized cost optimization strategies.
84. Machine learning engineers fine-tuned predictive models.
85. Cybersecurity teams deployed honeypots for threat detection.
86. Compliance officers reviewed ISO 27001 reports.
87. Cloud-native deployments minimized provisioning delays.
88. SaaS platforms supported dynamic scaling features.
89. DevOps pipelines included automated rollback triggers.
90. Predictive models updated in near real-time.
91. Serverless workflows executed event-driven tasks.
92. Knowledge graphs enabled semantic search capabilities.
93. Zero-shot learning handled unseen data categories.
94. Edge analytics processed sensor data locally.
95. AI pipelines maintained version-controlled training datasets.
96. Real-time metrics guided business decisions.
97. Cloud orchestration automated scaling policies.
98. Data visualization improved incident response times.
99. Enterprise teams collaborated across multiple regions.
100. AI-enhanced decision systems delivered competitive advantages."""
entities=extract_entities_no_spacy(string)
for ent in entities:
  label=ent["label"].lower()
  if label in ["name of a person", "organization"]:
    get_fake_value(label,ent["text"])
  print(f'entity: {ent["text"]}, label: {ent["label"]}')
mapping={
    "forward_mapping":forward_mapping,
    "backward_mapping":backward_mapping,

}

with open("descriptive_mapping.json","w", encoding="utf-8") as f:
    json.dump(mapping, f, ensure_ascii=False, indent=4)



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


entity: James Carter, label: name of a person
entity: TechNova Solutions, label: organization
entity: Maya Patel, label: name of a person
entity: NeoAI Labs, label: organization
entity: Michael Zhang, label: name of a person
entity: CloudSphere Technologies, label: organization
entity: Sophia Rodriguez, label: name of a person
entity: Sentinel Secure, label: organization
entity: David Kim, label: name of a person
entity: SkyGrid IT, label: organization
entity: Emma Johnson, label: name of a person
entity: DataBridge, label: organization
entity: DataBridge Consulting, label: organization
entity: Rajiv Menon, label: name of a person
entity: QuantumCore Systems, label: organization
entity: Olivia Brown, label: name of a person
entity: QuantumCore Systems, label: organization
entity: Ethan Hughes, label: name of a person
entity: QuantumCore Systems, label: organization
entity: Isabella Chen, label: name of a person
entity: InnovateX Labs, label: organization
entity: TechNova Solutions, lab

In [17]:
masked_string=mask_sentence(string)
print(masked_string)


1. Theresa Williams works at Williams-Waller Co as a Senior Cloud Engineer.
2. Ruth Jenkins is a Lead Data Scientist at Santiago-Lester LLC.
3. Frances Wagner is the Program Manager at Harris-Roman Group.
4. Kaitlin Myers is Director of Cybersecurity at Shaw-Campbell International.
5. Nicolas Stevens works as a Senior DevOps Specialist at Gibson-Kim Inc.
6. Donna Beard is a Business Analyst at Sampson-Gonzalez Inc.
7. Helen Sanchez serves as CTO of Cox-Kline Corporation.
8. Gavin Velazquez leads the AI Product Strategy division at Cox-Kline Corporation.
9. Jamie Stokes is a Solutions Architect at Cox-Kline Corporation.
10. Catherine Hicks is a Machine Learning Engineer at Osborne-Moreno LLC.
11. Williams-Waller Co provides cloud-native SaaS platforms.
12. Santiago-Lester LLC specializes in zero-shot classification research.
13. Harris-Roman Group develops machine learning pipelines.
14. Shaw-Campbell International builds real-time threat detection services.
15. Gibson-Kim Inc manages c

In [18]:
documents = masked_string.split("\n")
# --- Step 2: Load Embedding Model ---
model = SentenceTransformer('all-MiniLM-L6-v2')

# --- Step 3: Convert Documents to Embeddings ---
embeddings = model.encode(documents)
embeddings = np.array(embeddings, dtype=np.float32)

# --- Step 4: Build FAISS Vector Store ---
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

# --- Step 5: Retrieval Function ---
def retrieve(query, k=3):
    query_vector = model.encode([query])
    query_vector = np.array(query_vector, dtype=np.float32)
    distances, indices = index.search(query_vector, k)
    return [documents[i] for i in indices[0]]

# --- Step 6: Simple LLM Simulation ---
def generate_answer(query, context):
    # Simulate by concatenating context (since no LLM)
    return f"Context used:\n{unmask_summary(context)}\n\nAnswer: Based on the context, {query}"

# --- Step 7: RAG Query ---
def rag_query(query):
    masked_query=mask_sentence(query)
    print(f'masked query is :{masked_query}')
    retrieved_docs = retrieve(masked_query)
    context = "\n".join(retrieved_docs)
    return generate_answer(query, context)


In [21]:

# --- Test ---
print(rag_query("details about  kim"))

masked query is :details about  Lindsay Cohen
Context used:
10. Isabella Chen is a Machine Learning Engineer at InnovateX Labs.
61. Rajiv Menon invested in multi-cloud data orchestration.
28. Isabella Chen demonstrated customer support automation tools.

Answer: Based on the context, details about  kim
